In [ ]:
import json
import pandas as pd
import math
import numpy as np

In [ ]:
#CONSTANTS#
PATH = '/content/drive/My Drive/mrna_data/'
KNN_CANDIDATES = [1,2,3,8,10,13,15, 20, 25,27,28]

In [ ]:
# Load the Drive helper and mount
from google.colab import drive

# This will prompt for authorization.
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
#PREPROCESSING#
# define the triplet bases
def define_lang():
  lang = []
  amino = "UACG"
  for c1 in amino:
    for c2 in amino:
      for c3 in amino:
        lang.append(c1+c2+c3)
  lang = np.array(lang)

  return lang
#num_clusters = 1000

# convert each sequence of bases to a frequency count of the triplets
def seq_to_freq(seq):
  freq = []
  lang = define_lang()
  for word in lang:
    #print(seq.split(word))
    freq.append(len(seq.split(word))-1)
  freq = np.array(freq)
  return freq

"""def make_clusters(train):
  kmeans = KMeans(n_clusters=num_clusters, random_state=0).fit(train)
  return kmeans.labels_

def avg_clusters(labels, x_train, y_train):
  new_x_train = [[0.0]*64] * num_clusters
  new_x_train = np.array(new_x_train)
  new_y_train = [[0.0]*len(y_train[0])] * num_clusters
  new_y_train = np.array(new_y_train)

  for i in range(len(x_train)):
    new_x_train[labels[i]] = np.add(new_x_train[labels[i]], x_train[i])
  num_molec_per_cluster = len(x_train)/num_clusters
  for i in range(len(new_x_train)):
    new_x_train[i] = new_x_train[i]/num_molec_per_cluster

  for i in range(len(y_train)):
    new_y_train[labels[i]] = np.add(new_y_train[labels[i]], y_train[i])
  for i in range(len(new_y_train)):
    new_y_train[i] = new_y_train[i]/num_molec_per_cluster
  
  return new_x_train, new_y_train"""
  


'def make_clusters(train):\n  kmeans = KMeans(n_clusters=num_clusters, random_state=0).fit(train)\n  return kmeans.labels_\n\ndef avg_clusters(labels, x_train, y_train):\n  new_x_train = [[0.0]*64] * num_clusters\n  new_x_train = np.array(new_x_train)\n  new_y_train = [[0.0]*len(y_train[0])] * num_clusters\n  new_y_train = np.array(new_y_train)\n\n  for i in range(len(x_train)):\n    new_x_train[labels[i]] = np.add(new_x_train[labels[i]], x_train[i])\n  num_molec_per_cluster = len(x_train)/num_clusters\n  for i in range(len(new_x_train)):\n    new_x_train[i] = new_x_train[i]/num_molec_per_cluster\n\n  for i in range(len(y_train)):\n    new_y_train[labels[i]] = np.add(new_y_train[labels[i]], y_train[i])\n  for i in range(len(new_y_train)):\n    new_y_train[i] = new_y_train[i]/num_molec_per_cluster\n  \n  return new_x_train, new_y_train'

In [ ]:
#KNN#

# calculate the distance between two frequency vectors
def dist(freq1, freq2):
  dist = np.linalg.norm(freq1-freq2)
  return dist

# find the kNN for a single frequency vector
def find_knn(x_test_vector, x_train, knn):
  # dictionary containing keys as distances from x_test_vector to every vector in x_train,
  # and values as the indices of the vectors in x_train that correspond to these distances
  distances = {}

  # find distance between x_test_vector and every vector in x_train
  for vec_idx in range(len(x_train)):
    d = dist(x_test_vector, x_train[vec_idx])

    # record distance and index in the dictionary 
    if d in distances:
      distances[d].append(vec_idx)
    else:
      distances[d] = [vec_idx]

  # sort the distances from least to greatest
  sorted_distances = list(distances.keys())
  sorted_distances.sort()

  # get the kNN for x_test_vector
  knn_arr = []

  # record indices corresponding to first knn distances 
  for i in range(knn):
    d = sorted_distances[i]
    for idx in distances[d]:
      knn_arr.append(idx)
  # truncate knn in case multiple indices correspond to one distance
  knn_arr = knn_arr[:knn]

  return knn_arr 

def predict(x_test, x_train, y_train, knn):
  '''
  parameters:
    x_test: array of frequency vectors to be predicted
    x_train: array of frequency vectors from which neighbors will be found
    y_train: array of reactivity values corresponding to each frequency vector
    knn: number of neighbors to find
  '''
  y_pred = []

  # predict reactivity values for each vector in x_test 
  for vector in x_test:
    knn_arr = find_knn(vector, x_train, knn)

    # calculate prediction
    y_pred_vector = np.zeros((len(y_train[0]), 3))
    # sum the reactivities in each neighbor
    for idx in knn_arr:
      y_pred_vector = np.add(y_pred_vector, y_train[idx])
    # divide each sum by knn to get the average reactivity
    y_pred_vector = np.divide(y_pred_vector, knn)
    
    # store this prediction in the prediction vector
    y_pred.append(y_pred_vector)

  return y_pred

# calculate error
def error(y_pred, y_test, metric='MCRMSE'):
  err = 0
  if metric == 'MSE':
    mse = 0

    # sum the squared differences of each prediction
    for vector_idx in range(len(y_pred)):
      # for each reactivity vector, find the squared differences of each prediction
      diff_square_vector = np.subtract(y_test[vector_idx], y_pred[vector_idx])
      diff_square_vector = np.power(diff_square_vector, 2)

      # sum the values of these squared differences and add to mse
      mse += np.sum(diff_square_vector)

    # divide by total number of predictions
    mse /= (len(y_pred) * len(y_pred[0]))
    err = mse
  if metric == "MCRMSE":
    #mean columnwise root mean squared error)#
    #error specified by kaggle competition, accounts for all five categories being measured#
    mcrmse = 0

    y_pred_t = np.transpose(np.array(y_pred), (2, 0, 1))
    y_test_t = np.transpose(y_test, (2, 0, 1))
    for i in range(3):
      mse = error(y_pred_t[i], y_test_t[i], 'MSE')
      mcrmse += mse**0.5

    # divide by total number of prediction groups
    err = mcrmse / 3


  return err

In [ ]:
def find_best_k(x_train, x_test, y_train, y_test):
  best_k = 0
  best_err = 100

  for k in KNN_CANDIDATES:
    y_pred = predict(x_test, x_train, y_train, k)
    err = error(y_pred, y_test)

    if err < best_err:
      best_k = k
      best_err = err

    print(k, err)

  return best_k

In [ ]:
# read in the raw data and define the test and training datasets 
def load_raw_data(path):
  train_raw = pd.read_json(path+'train.json', lines=True)
  train = train_raw[0:1800]
  test = train_raw[1800:2400]
  x_train_seq = train['sequence']
  x_test_seq = test['sequence']
  #y_train = [train['reactivity'], train['deg_Mg_pH10'], train['deg_pH10'], train['deg_Mg_50C'], train['deg_50C']]
  #y_test = [test['reactivity'], test['deg_Mg_pH10'], test['deg_pH10'], test['deg_Mg_50C'], test['deg_50C']]
  y_train = [train['reactivity'], train['deg_Mg_pH10'], train['deg_pH10']]
  y_test = [test['reactivity'], test['deg_Mg_pH10'], test['deg_pH10']]
  y_train = np.array(y_train)
  y_test = np.array(y_test)
  y_train = y_train.transpose((1,2,0))
  y_test = y_test.transpose((1,2,0))
  y_train_reacterr = train_raw[0:1800]['reactivity_error']
  y_train_reacterr = np.array(y_train_reacterr)

  return x_train_seq, x_test_seq, y_train, y_test, y_train_reacterr

# convert each sequence feature to a frequency vector
def process_data(x_train_seq, x_test_seq):
  x_train = []
  x_test = []
  for seq in x_train_seq:
    x_train.append(seq_to_freq(seq))
  for seq in x_test_seq:
    x_test.append(seq_to_freq(seq))
  x_train = np.array(x_train)
  x_test = np.array(x_test)
  #print(len(seq_to_freq(train['sequence'][0])))
  #print(x_train[0])

  return x_train, x_test

# adjust y_train reactivity values according to corresponding error (aka weighting)
# UNIMPLEMENTED
def process_data_y(y_train, y_train_reacterr):
  return y_train

In [ ]:
if __name__ == '__main__':
  x_train_seq, x_test_seq, y_train, y_test, y_train_reacterr = load_raw_data(PATH)
  #print(y_train.shape)
  #print(x_train.shape)
  x_train, x_test = process_data(x_train_seq, x_test_seq)
  y_train = process_data_y(y_train, y_train_reacterr)

  best_k = find_best_k(x_train, x_test, y_train, y_test)

  print(best_k)

  # x_test_unlabeled = load_test_data(PATH)
  # y_pred = predict(x_test_unlabeled, x_train, y_train, best_k)

  

1 1.1834443177124847
2 1.029025056694351
3 0.9523750740692599
8 0.8814729037884628
10 0.8715021135791471
13 0.8610573999199985
15 0.8562489792092149
20 0.8490626303001685
25 0.8486649184222247
27 0.8487833763354912
28 0.8489864492368765
25


In [ ]:
# print(labeled_x_train)
# print(len(small_x_train))
# print(small_x_train[99])
# print(len(small_y_train))
# for molec in small_y_train:
#   for pos in molec:
#     if(pos>100):
#       print(pos)